Norint paleisti toliau pateiktus užrašų knygelius, jei dar to nepadarėte, turite įdiegti modelį, kuris naudoja `text-embedding-ada-002` kaip pagrindinį modelį, ir nustatyti diegimo pavadinimą .env faile kaip `AZURE_OPENAI_EMBEDDINGS_ENDPOINT`


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-05-15"
  )

model = os.environ['AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT']

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

Toliau įkelsime įterpimų indeksą į Pandas duomenų rėmelį. Įterpimų indeksas saugomas JSON faile, pavadintame `embedding_index_3m.json`. Įterpimų indeksas talpina įterpimus kiekvienam YouTube transkriptui iki 2023 m. spalio pabaigos.


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

Toliau sukursime funkciją pavadinimu `get_videos`, kuri ieškos Embedding indekse pagal užklausą. Funkcija grąžins 5 geriausiai užklausai panašius vaizdo įrašus. Funkcija veikia taip:

1. Pirmiausia sukuriama Embedding indekso kopija.
2. Tada apskaičiuojamas užklausos Embeddingas naudojant OpenAI Embedding API.
3. Tuomet Embedding indekse sukuriamas naujas stulpelis pavadinimu `similarity`. Stulpelyje `similarity` yra kosininio panašumo reikšmė tarp užklausos Embeddingo ir kiekvieno vaizdo segmento Embeddingo.
4. Toliau Embedding indeksas filtruojamas pagal stulpelį `similarity`. Filtruojami tik tie vaizdo įrašai, kurių kosininis panašumas yra didesnis arba lygus 0.75.
5. Galiausiai Embedding indeksas rūšiuojamas pagal stulpelį `similarity` ir grąžinami 5 geriausi vaizdo įrašai.


In [ ]:
def cosine_similarity(a, b):
    if len(a) > len(b):
        b = np.pad(b, (0, len(a) - len(b)), 'constant')
    elif len(b) > len(a):
        a = np.pad(a, (0, len(b) - len(a)), 'constant')
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

Ši funkcija yra labai paprasta, ji tiesiog išveda paieškos užklausos rezultatus.


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. Pirmiausia, įterpimo indeksas įkeliamas į Pandas duomenų rėmelį.
2. Tada vartotojui pateikiama užklausa įvesti.
3. Po to kviečiama funkcija `get_videos`, kuri ieško įterpimo indekse pagal užklausą.
4. Galiausiai kviečiama funkcija `display_results`, kuri rodo rezultatus vartotojui.
5. Vartotojui vėl pateikiama užklausa įvesti. Šis procesas tęsiasi tol, kol vartotojas įveda `exit`.

![](../../../../translated_images/lt/notebook-search.1e320b9c7fcbb0bc.png)

Jums bus pasiūlyta įvesti užklausą. Įveskite užklausą ir paspauskite enter. Programa pateiks sąrašą vaizdo įrašų, kurie yra susiję su užklausa. Programa taip pat pateiks nuorodą į vietą vaizdo įraše, kur randamas atsakymas į klausimą.

Štai keletas užklausų, kurias galite išbandyti:

- Kas yra Azure Machine Learning?
- Kaip veikia konvoliuciniai neuroniniai tinklai?
- Kas yra neuroninis tinklas?
- Ar galiu naudoti Jupyter užrašų knygutes su Azure Machine Learning?
- Kas yra ONNX?


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from input
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Atsakomybės apribojimas**:
Šis dokumentas buvo išverstas naudojant dirbtinio intelekto vertimo paslaugą [Co-op Translator](https://github.com/Azure/co-op-translator). Nors stengiamės užtikrinti tikslumą, prašome atkreipti dėmesį, kad automatiniai vertimai gali turėti klaidų ar netikslumų. Originalus dokumentas gimtąja kalba turėtų būti laikomas autoritetingu šaltiniu. Svarbiai informacijai rekomenduojamas profesionalus žmogaus vertimas. Mes neatsakome už bet kokius nesusipratimus ar neteisingus aiškinimus, kilusius dėl šio vertimo naudojimo.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
